# Creating a custom Location Costmap

In this tutorial we will walk through the creation of a costmap through machine learning.
First we need to gather a lot of data. For that we will write a randomized experiment for grasping a couple of objects.
In the experiment the robot will try to grasp a randomized object using random poses and torso heights.

In [1]:
!pip install --upgrade probabilistic_model

Requirement already up-to-date: probabilistic_model in /home/dprueser/.local/lib/python3.8/site-packages (1.6.20)


In [2]:
from pycram.designators.actions.actions import ParkArmsActionPerformable, MoveTorsoActionPerformable
from tf import transformations
import itertools
from typing import Optional, List, Tuple
import numpy as np
import sqlalchemy.orm
import tqdm
import pycram.orm.base
import pycram.task
from pycram.bullet_world import BulletWorld, Object as BulletWorldObject
from pycram.designators.action_designator import MoveTorsoAction, PickUpAction, NavigateAction, ParkArmsAction
from pycram.designators.object_designator import ObjectDesignatorDescription
import pycram.enums
from pycram.plan_failures import PlanFailure
from pycram.process_module import ProcessModule
from pycram.process_module import simulated_robot
import pycram.orm
from pycram.orm.base import Position, RobotState
from pycram.orm.task import TaskTreeNode, Code
from pycram.orm.action_designator import PickUpAction as ORMPickUpAction 
from pycram.orm.object_designator import Object 
import sqlalchemy.sql
import pandas as pd
from pycram.pose import Pose

np.random.seed(420)

ProcessModule.execution_delay = False
pycram.orm.base.ProcessMetaData().description = "Tutorial for learning from experience in a Grasping action."


class GraspingExplorer:
    """Class to try randomized grasping plans."""

    world: Optional[BulletWorld]

    def __init__(self, robots: Optional[List[Tuple[str, str]]] = None, objects: Optional[List[Tuple[str, str]]] = None,
                 arms: Optional[List[str]] = None, grasps: Optional[List[str]] = None,
                 samples_per_scenario: int = 1000):
        """
        Create a GraspingExplorer.
        :param robots: The robots to use
        :param objects: The objects to try to grasp
        :param arms: The arms of the robot to use
        :param grasps: The grasp orientations to use
        :param samples_per_scenario: The number of tries per scenario.
        """
        # store exploration space
        if not robots:
            self.robots: List[Tuple[str, str]] = [("pr2", "pr2.urdf")]

        if not objects:
            self.objects: List[Tuple[str, pycram.enums.ObjectType, str]] = [("cereal", pycram.enums.ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl"), 
                                                                            ("bowl", pycram.enums.ObjectType.BOWL, "bowl.stl"),
                                                                            ("milk", pycram.enums.ObjectType.MILK, "milk.stl"),
                                                                            ("spoon", pycram.enums.ObjectType.SPOON, "spoon.stl")]

        if not arms:
            self.arms: List[str] = ["left", "right"]

        if not grasps:
            self.grasps: List[str] = ["left", "right", "front", "top"]

        # store trials per scenario
        self.samples_per_scenario: int = samples_per_scenario

        # chain hyperparameters
        self.hyper_parameters = [self.robots, self.objects, self.arms, self.grasps]

        self.total_tries = 0
        self.total_failures = 0

    def perform(self, session: sqlalchemy.orm.Session):
        """
        Perform all experiments.
        :param session: The database-session to insert the samples in.
        """

        # create progress bar
        progress_bar = tqdm.tqdm(
            total=np.prod([len(p) for p in self.hyper_parameters]) * self.samples_per_scenario)
        
        self.world = BulletWorld("DIRECT")

        # for every robot
        for robot, robot_urdf in self.robots:

            # spawn it
            robot = BulletWorldObject(robot, pycram.enums.ObjectType.ROBOT, robot_urdf)

            # for every obj
            for obj, obj_type, obj_stl in self.objects:

                # spawn it
                bw_object = BulletWorldObject(obj, obj_type, obj_stl, Pose([0, 0, 0.75], [0, 0, 0, 1]))

                # create object designator
                object_designator = ObjectDesignatorDescription(names=[obj])

                # for every arm and grasp pose
                for arm, grasp in itertools.product(self.arms, self.grasps):
                    # sample positions in 2D
                    positions = np.random.uniform([-2, -2], [2, 2], (self.samples_per_scenario, 2))

                    # for every position
                    for position in positions:

                        # set z axis to 0
                        position = [*position, 0]

                        # calculate orientation for robot to face the object
                        angle = np.arctan2(position[1], position[0]) + np.pi
                        orientation = list(transformations.quaternion_from_euler(0, 0, angle, axes="sxyz"))

                        # try to execute a grasping plan
                        with simulated_robot:

                            ParkArmsActionPerformable(pycram.enums.Arms.BOTH).perform()
                            # navigate to sampled position
                            NavigateAction([Pose(position, orientation)]).resolve().perform()

                            # move torso
                            height = np.random.uniform(0., 0.33, 1)[0]
                            MoveTorsoActionPerformable(height).perform()

                            # try to pick it up
                            try:
                                PickUpAction(object_designator, [arm], [grasp]).resolve().perform()

                            # if it fails
                            except PlanFailure:

                                # update failure stats
                                self.total_failures += 1

                            # reset BulletWorld
                            self.world.reset_bullet_world()

                            # update progress bar
                            self.total_tries += 1

                            # insert into database
                            pycram.task.task_tree.insert(session, use_progress_bar=False)
                            pycram.task.reset_tree()

                            progress_bar.update()
                            progress_bar.set_postfix(success_rate=(self.total_tries - self.total_failures) /
                                                                  self.total_tries)

                bw_object.remove()
            robot.remove()

        self.world.exit()

pybullet build time: May 20 2022 19:44:17
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
[WARN] [1706544338.060994]: Failed to import Giskard messages
[WARN] [1706544338.065403]: Could not import RoboKudo messages, RoboKudo interface could not be initialized


Next we have to establish a connection to a database and execute the experiment a couple of times. Note that the (few) number of samples we generate is only for demonstrations.
For robust and reliable machine learning millions of samples are required.

In [3]:
engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:")
session = sqlalchemy.orm.Session(bind=engine)
pycram.orm.base.Base.metadata.create_all(bind=engine)
session.commit()

explorer = GraspingExplorer(samples_per_scenario=30)
explorer.perform(session)

  0%|          | 0/960 [00:00<?, ?it/s]Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name=

Write the database to a file if wanted.

In [4]:
import os
con = engine.raw_connection()

with open(os.path.join(os.path. expanduser('~'), "Documents", "costmap.dump"), 'w') as f:
    for line in con.iterdump():
        f.write('%s\n' % line)

Read the database from a file instead of executing all the experiments.

In [5]:
import os

engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:")
session = sqlalchemy.orm.Session(bind=engine)
# pycram.orm.base.Base.metadata.create_all(bind=engine)
session.commit()

with engine.connect() as con:
    with open(os.path.join(os.path. expanduser('~'), "Documents", "costmap.dump")) as file:
        raw_connection = engine.raw_connection()
        # raw_cursor = raw_connection()
        raw_connection.executescript(file.read())

Next we will create a dataframe from the previously created database by joining the relevant information together using the pycram.orm package.
We will get the following columns:
    - status of the grasping task
    - type of the grasped object
    - arm used to grasp
    - grasp pose
    - torso height relative to the object
    - x and y coordinate of the robot relative to the object.


In [6]:
from sqlalchemy import select
from pycram.orm.base import Pose as ORMPose

robot_pose = sqlalchemy.orm.aliased(ORMPose)
object_pose = sqlalchemy.orm.aliased(ORMPose)
robot_pos = sqlalchemy.orm.aliased(Position)
object_pos = sqlalchemy.orm.aliased(Position)

query = (select(TaskTreeNode.status, Object.type, ORMPickUpAction.arm, ORMPickUpAction.grasp,
                       sqlalchemy.label("relative torso height", object_pos.z - RobotState.torso_height),
                       sqlalchemy.label("x", robot_pos.x - object_pos.x), 
                       sqlalchemy.label("y", robot_pos.y - object_pos.y))
         .join(TaskTreeNode.code)
         .join(Code.designator.of_type(ORMPickUpAction))
         .join(ORMPickUpAction.robot_state)
         .join(robot_pose, RobotState.pose)
         .join(robot_pos, robot_pose.position)
         .join(ORMPickUpAction.object)
         .join(object_pose, Object.pose)
         .join(object_pos, object_pose.position))
print(query)

df = pd.read_sql(query, session.get_bind())
df["status"] = df["status"].apply(lambda x: str(x.name))
print(df)

SELECT "TaskTreeNode".status, "Object".type, "PickUpAction".arm, "PickUpAction".grasp, "Position_1".z - "RobotState".torso_height AS "relative torso height", "Position_2".x - "Position_1".x AS x, "Position_2".y - "Position_1".y AS y 
FROM "TaskTreeNode" JOIN "Code" ON "Code".id = "TaskTreeNode".code_id JOIN ("Designator" JOIN "Action" ON "Designator".id = "Action".id JOIN "PickUpAction" ON "Action".id = "PickUpAction".id) ON "Designator".id = "Code".designator_id JOIN "RobotState" ON "RobotState".id = "Action".robot_state_id JOIN "Pose" AS "Pose_1" ON "Pose_1".id = "RobotState".pose_id JOIN "Position" AS "Position_2" ON "Position_2".id = "Pose_1".position_id JOIN "Object" ON "Object".id = "PickUpAction".object_id JOIN "Pose" AS "Pose_2" ON "Pose_2".id = "Object".pose_id JOIN "Position" AS "Position_1" ON "Position_1".id = "Pose_2".position_id
        status                         type    arm grasp  \
0       FAILED  ObjectType.BREAKFAST_CEREAL   left  left   
1       FAILED  ObjectTyp

Next we will define a joint probability distributions over these points using, for instance, a Joint Probability Tree.

In [7]:
import jpt

model = jpt.trees.JPT(variables=jpt.infer_from_dataframe(df, scale_numeric_types=False, precision=0.05),
                      min_samples_leaf=10, min_impurity_improvement=0.01)
model.fit(df)

100%|██████████| 960/960 [01:43<00:00,  9.32it/s, success_rate=0.0854]


<JPT #innernodes = 70, #leaves = 71 (141 total)>

Finally, we will insert our model in the existing JPT-Costmap wrapper, create a toy simulation and visualize our result.

In [8]:
from pycram.designators.actions.actions import NavigateActionPerformable, PickUpActionPerformable
from pycram.resolver import JPTCostmapLocation


ProcessModule.execution_delay = True

world = BulletWorld("GUI")
robot = BulletWorldObject("pr2", pycram.enums.ObjectType.ROBOT, "pr2.urdf")

cereal = BulletWorldObject("milk", pycram.enums.ObjectType.MILK, "milk.stl", pose=Pose([1, 0, 0.75], [0,0,0,1]))
np.random.seed(420)
cml = JPTCostmapLocation(cereal, reachable_for=robot, model=model)
cml.visualize()
with simulated_robot:
    for sample in iter(cml):
        ParkArmsActionPerformable(pycram.enums.Arms.BOTH).perform()
        NavigateActionPerformable(sample.pose).perform()
        MoveTorsoActionPerformable(sample.torso_height).perform()
        try:
            PickUpActionPerformable(ObjectDesignatorDescription(types=[pycram.enums.ObjectType.MILK]).resolve(), arm=sample.reachable_arm, grasp=sample.grasp).perform()
            # time.sleep(5)
        except pycram.plan_failures.PlanFailure as p:
            continue
        break
world.exit()

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


Congratulations, you made the PR2 learn how to grasp objects.
Yeehaaw!🤠